# 실습 3: Vector Database 구성

## 목표
- S3 Vector Database 생성
- 소방서 데이터 임베딩
- Vector Search 테스트

## 3-1. 환경 설정

In [ ]:
from dotenv import load_dotenv
import os
import boto3
import json
import pandas as pd

load_dotenv()

REGION = os.getenv('AWS_REGION', 'us-west-2')
BUCKET_NAME = "firestation-location-xy"
INDEX_NAME = "fire-station"

print(f"🌍 Region: {REGION}")
print(f"🗄️ Bucket: {BUCKET_NAME}")
print(f"📊 Index: {INDEX_NAME}")

## 3-2. S3 Vector Bucket 생성

In [ ]:
s3vectors = boto3.client('s3vectors', region_name=REGION)

try:
    response = s3vectors.create_vector_bucket(
        vectorBucketName=BUCKET_NAME
    )
    print(f"✅ Vector Bucket 생성: {BUCKET_NAME}")
    print(f"📝 Bucket ARN: {response.get('vectorBucketArn', 'N/A')}")
except Exception as e:
    if 'BucketAlreadyExists' in str(e) or 'BucketAlreadyOwnedByYou' in str(e):
        print(f"ℹ️ Vector Bucket 이미 존재: {BUCKET_NAME}")
    else:
        print(f"⚠️ 오류: {e}")

## 3-3. Vector Index 생성

In [ ]:
s3vectors = boto3.client('s3vectors', region_name=REGION)

try:
    response = s3vectors.create_index(
        vectorBucketName=BUCKET_NAME,
        indexName=INDEX_NAME,
        dataType='float32',
        dimension=1024,
        distanceMetric='cosine'
    )
    print(f"✅ Vector Index 생성: {INDEX_NAME}")
    print(f"📝 Index ARN: {response.get('indexArn', 'N/A')}")
except Exception as e:
    if 'IndexAlreadyExists' in str(e):
        print(f"ℹ️ Index 이미 존재: {INDEX_NAME}")
    else:
        print(f"⚠️ 오류: {e}")

## 3-4. 소방서 데이터 로드

In [ ]:
import os

# 프로젝트 루트 디렉토리 확인
project_root = os.getcwd()
csv_path = os.path.join(project_root, 'misc', 'fire_station.csv')

# 파일 존재 확인
if not os.path.exists(csv_path):
    print(f"⚠️ 파일을 찾을 수 없습니다: {csv_path}")
    print(f"📂 현재 디렉토리: {project_root}")
    print(f"\n💡 해결 방법:")
    print(f"   1. misc/ 디렉토리에 CSV 파일이 있는지 확인")
    print(f"   2. 또는 아래 경로를 수정하세요")
else:
    print(f"📂 CSV 파일 읽기: {csv_path}")
    df = pd.read_csv(csv_path, encoding='euc-kr')
    
    print(f"✅ 총 {len(df)}개 소방서 데이터 로드")
    print(f"\n📊 데이터 샘플:")
    display(df.head())

## 3-5. 소방서 데이터 임베딩 및 저장

In [ ]:
bedrock = boto3.client("bedrock-runtime", region_name=REGION)

print(f"🚀 임베딩 시작 (예상: {len(df) * 0.5 / 60:.1f}분)\n")

vectors = []
batch_size = 50

for idx, row in df.iterrows():
    latitude = float(row['X좌표'])
    longitude = float(row['Y좌표'])
    
    text = f"{row['소방서 및 안전센터명']} , 위치: {row['주소']} , 소속: {row['상위 본부명']} , 위도: {latitude:.6f} , 경도: {longitude:.6f}"
    
    if (idx + 1) % 10 == 0:
        print(f"처리중 [{idx+1}/{len(df)}]: {row['소방서 및 안전센터명']}")
    
    response = bedrock.invoke_model(
        modelId="amazon.titan-embed-text-v2:0",
        body=json.dumps({
            "inputText": text,
            "dimensions": 1024,
            "normalize": True
        })
    )
    
    embedding = json.loads(response["body"].read())["embedding"]
    
    vectors.append({
        "key": f"firestation_{idx}",
        "data": {"float32": embedding},
        "metadata": {
            "name": str(row['소방서 및 안전센터명']),
            "address": str(row['주소']),
            "phone": str(row['전화번호']),
            "latitude": str(latitude),
            "longitude": str(longitude),
            "type": str(row['유형']),
            "headquarters": str(row['상위 본부명'])
        }
    })
    
    if len(vectors) == batch_size:
        s3vectors.put_vectors(
            vectorBucketName=BUCKET_NAME,
            indexName=INDEX_NAME,
            vectors=vectors
        )
        print(f"✅ {idx+1}개 저장 완료")
        vectors = []

if vectors:
    s3vectors.put_vectors(
        vectorBucketName=BUCKET_NAME,
        indexName=INDEX_NAME,
        vectors=vectors
    )

print(f"\n🎉 임베딩 완료!")

## 3-6. Vector Search 테스트

In [ ]:
test_address = "서울특별시 서초구 방배중앙로 06681"

print(f"🔍 테스트 주소: {test_address}\n")

# 주소 임베딩
embedding_response = bedrock.invoke_model(
    modelId="amazon.titan-embed-text-v2:0",
    body=json.dumps({
        "inputText": test_address,
        "dimensions": 1024,
        "normalize": True
    })
)

query_embedding = json.loads(embedding_response["body"].read())["embedding"]

# Vector Search
search_response = s3vectors.query_vectors(
    vectorBucketName=BUCKET_NAME,
    indexName=INDEX_NAME,
    queryVector={"float32": query_embedding},
    topK=5,
    returnMetadata=True
)

print(f"✅ 가까운 소방서 5곳:\n")
for idx, item in enumerate(search_response["vectors"], 1):
    meta = item.get("metadata", {})
    print(f"{idx}. {meta.get('name')}")
    print(f"   위도: {meta.get('latitude')}, 경도: {meta.get('longitude')}")
    print(f"   주소: {meta.get('address')}")
    print(f"   전화: {meta.get('phone')}")

## ✅ 실습 3 완료!

### 생성된 리소스
- S3 Bucket: `firestation-location-xy`
- Vector Index: `fire-station`
- 임베딩된 소방서 데이터

### 다음 단계
👉 **lab4_agent_runtime.ipynb**